In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import string
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/sarcasm/sarcasm_tam_train.csv")

In [ ]:
df.loc[2278]

Text      I'm waiting karthi fans hit like
labels                       Non-sarcastic
Name: 2278, dtype: object

In [ ]:
df['Text'] = df['Text'].replace(r'(?i)@\S+', '', regex=True)
#Remove punctuation marks
df['Text'] = df['Text'].str.replace('(?i)['+string.punctuation+']', '', regex=True)
#Convert the entire text to lowercase
df['Text'] = df['Text'].apply(lambda x: x.lower())
#Remove the http links and emojis
df['Text'] = df['Text'].replace(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", regex=True)
#Remove all the numbers from the tweet column
df['Text'] = df['Text'].str.replace('\d+', '', regex=True)

'df[\'Text\'] = df[\'Text\'].replace(r\'(?i)@\\S+\', \'\', regex=True)\n#Remove punctuation marks\ndf[\'Text\'] = df[\'Text\'].str.replace(\'(?i)[\'+string.punctuation+\']\', \'\', regex=True)\n#Convert the entire text to lowercase\ndf[\'Text\'] = df[\'Text\'].apply(lambda x: x.lower())\n#Remove the http links and emojis\ndf[\'Text\'] = df[\'Text\'].replace(r"(@\\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\\w+:\\/\\/\\S+)|^rt|http.+?", "", regex=True)\n#Remove all the numbers from the tweet column\ndf[\'Text\'] = df[\'Text\'].str.replace(\'\\d+\', \'\', regex=True)'

In [ ]:
df

,Text,labels
0,அர்யவுக்கு ஒரு நல்ல வாய்ப்பு சிங்கம் சூரியா அ...,Non-sarcastic
1,பள்ளியோ கல்லூரியோ படித்துக்கொண்டிருக்கும்போது ...,Non-sarcastic
2,தல தல தல தல தல தல தல தல தல தல தல தல தல தல தல த...,Non-sarcastic
3,All the best to the team....super ah na Oru ka...,Non-sarcastic
4,"Bahut over mack up kiya hua hai, it is look li...",Non-sarcastic
...,...,...
29565,Ajth sar mass vasanam Ner konda baarvai mass,Non-sarcastic
29566,Adei anga ENPT trailer mukkitu iruku innum,Sarcastic
29567,KV ANANDH SIR SENTIMENT ayan maatran kavan ma...,Non-sarcastic
29568,ஒவ்வொருத்தருக்கும் ஒவ்வொரு பைத்தியம். நமக்கு ய...,Non-sarcastic


In [ ]:
df[df['labels']=='Non-sarcastic']

,Text,labels
0,அர்யவுக்கு ஒரு நல்ல வாய்ப்பு சிங்கம் சூரியா அ...,Non-sarcastic
1,பள்ளியோ கல்லூரியோ படித்துக்கொண்டிருக்கும்போது ...,Non-sarcastic
2,தல தல தல தல தல தல தல தல தல தல தல தல தல தல தல த...,Non-sarcastic
3,All the best to the team....super ah na Oru ka...,Non-sarcastic
4,"Bahut over mack up kiya hua hai, it is look li...",Non-sarcastic
...,...,...
29564,2019 La super social Message Thala Vera level ...,Non-sarcastic
29565,Ajth sar mass vasanam Ner konda baarvai mass,Non-sarcastic
29567,KV ANANDH SIR SENTIMENT ayan maatran kavan ma...,Non-sarcastic
29568,ஒவ்வொருத்தருக்கும் ஒவ்வொரு பைத்தியம். நமக்கு ய...,Non-sarcastic


In [ ]:
df[df['labels']=='Sarcastic']

,Text,labels
7,Avara Contol Pannunga Pls... Vera level expre...,Sarcastic
12,Family sentiment... Pondatti pullainga... Enga...,Sarcastic
13,ASURAN trailer .... Kollla massu.... #asuran #...,Sarcastic
19,Dai intha padatha eppada release pannuvangada,Sarcastic
20,2:01 ( sound missing ) ennada ivlo kodi potu p...,Sarcastic
...,...,...
29554,உதயநிதிய தவிர பாக்கி எல்லாரும் frame ல merge ஆ...,Sarcastic
29558,Enaya ipdi use panirkinga super starah sema,Sarcastic
29560,marana mass full an full vera leval entry.....,Sarcastic
29563,Ethu vanthalum final nangatha (viswasam) intha...,Sarcastic


In [ ]:
df.loc[2955]

In [ ]:
def preprocess_text(text):
    # Convert English text to lowercase
    text = re.sub(r'[A-Za-z]+', lambda x: x.group(0).lower(), text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove punctuations (excluding Tamil characters)
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', text)

    # Remove emojis
    text = re.sub(r'[^\w\s\u0B80-\u0BFF]', '', text, flags=re.UNICODE)

    return text

# Apply preprocessing to the 'Text' column
df['Text'] = df['Text'].apply(preprocess_text)

# Display the first few rows of the dataframe to verify preprocessing
df

,Text,labels
0,அர்யவுக்கு ஒரு நல்ல வாய்ப்பு சிங்கம் சூரியா அ...,Non-sarcastic
1,பள்ளியோ கல்லூரியோ படித்துக்கொண்டிருக்கும்போது ...,Non-sarcastic
2,தல தல தல தல தல தல தல தல தல தல தல தல தல தல தல த...,Non-sarcastic
3,all the best to the teamsuper ah na oru kadhai...,Non-sarcastic
4,bahut over mack up kiya hua hai it is look lik...,Non-sarcastic
...,...,...
29565,ajth sar mass vasanam ner konda baarvai mass,Non-sarcastic
29566,adei anga enpt trailer mukkitu iruku innum,Sarcastic
29567,kv anandh sir sentiment ayan maatran kavan ma...,Non-sarcastic
29568,ஒவ்வொருத்தருக்கும் ஒவ்வொரு பைத்தியம் நமக்கு யு...,Non-sarcastic


In [ ]:
df.loc[2280]

Text      అననయయ తలగ ల కడ చయడ అననయయ మవస పలజ అననయయ
labels                             Non-sarcastic
Name: 2280, dtype: object

In [ ]:
!pip install polyglot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52561 sha256=49ec5dd90b5cfd8f938202934a61bf3be37588b4c88f9928f325e2f44c05ef80
  Stored in directory: /root/.cache/pip/wheels/aa/92/4a/b172589446ba537db3bdb9a1f2204f27fe71217981c14ac368
Successfully built polyglot


In [ ]:
!pip install pyicu pycld2 morfessor
!polyglot download embeddings2.ml  # Malayalam
!polyglot download embeddings2.te  # Telugu
!polyglot download embeddings2.en  # English
!polyglot download embeddings2.ta  # Tamil
!polyglot download embeddings2.fr  # French
!polyglot download embeddings2.hi  # Hindi
!polyglot download embeddings2.es  # Spanish
# Add more languages as needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyicu: filename=PyICU-2.13.1-cp310-cp310-linux_x86_64.whl size=1801401 sha256=59d66a4ba533be9dac3a87c45647b710a95ee88127fbc14741dc49be3c0bd008
  Stored in directory: /root/.cache/pip/wheels/8e/45/08/f4bc505e31eb7bb779d493226921abf18cf55ed30142e70eae
  Created wheel for pycld2: filename=pycld2-0.41-cp310-cp310-linux_x86_64.whl size=9904072 sha256=d5fb5def08c349a8b067bac40a63c59c882d1398327614f4c34452e129b10685
  Stored in directory: /root/.cache/pip/wheels/be/81/31/240c89c845e008a93d98542325270007de595bfd356eb0b06c
Successfully built pyicu pycld2
[polyglot_data] Downloading package embeddings2.ml to
[polyglot_data]     /root/polyglot_

In [ ]:
from polyglot.detect import Detector

def ling(x):
   # Handle empty or very short strings
   if x is None or len(x.strip()) < 10:  # Adjust the threshold as needed
       return 'unknown'
   try:
       detector = Detector(x)
       return detector.language.name
   except:
       return 'unknown'  # Return 'unknown' if language detection fails

df['lang'] = df['Text'].apply(lambda x: ling(x))

Streaming output truncated to the last 5000 lines.


In [ ]:
l=df['lang'].unique()
for x in l:
  print(x)

Tamil
English
Nyanja
Malay
Khasi
Quechua
Manx
Azerbaijani
Turkish
Swahili
Danish
Sanskrit
Norwegian Nynorsk
Kinyarwanda
Oromo
Klingon
Xhosa
Indonesian
Estonian
Finnish
Zulu
Spanish
Scottish Gaelic
Galician
zzp
Rundi
Lithuanian
Javanese
Scots
Wolof
Tagalog
Slovak
Afar
Venda
Shona
Fijian
Aymara
Somali
Esperanto
Uzbek
Interlingue
Corsican
unknown
Breton
Igbo
Kalaallisut
Ganda
Slovenian
Afrikaans
Malagasy
Tswana
Latin
German
Hausa
Yoruba
Seselwa Creole French
Icelandic
Serbian
Irish
Swedish
Albanian
Turkmen
Faroese
Arabic
Nauru
Welsh
Hungarian
Lingala
Czech
Samoan
Romanian
Norwegian
Bosnian
Basque
Romansh
Swati
Tsonga
Māori
Bislama
Akan
Croatian
Sundanese
Dutch
Telugu
Luxembourgish
Tongan
Hmong
Portuguese
Waray
Occitan
Cebuano
French
Italian
Polish
Hawaiian
Malayalam
Guarani
Sango
Volapük
Haitian Creole
Latvian
Interlingua
Kannada
Inupiaq
Catalan
Russian
Maltese
Tatar
Southern Sotho
Morisyen
Zhuang
Northern Sotho
Urdu
Greek
Vietnamese
Bhojpuri
Hindi
Gujarati
Sindhi
Western Frisian


In [ ]:
df[df['lang']=='Urdu']


,Text,labels,lang
12172,شسشجظ د46صقك ظض ض كيم كشسسذكظظظدموردي ظفر صء س...,Non-sarcastic,Urdu


In [ ]:
df.drop(df[df['lang']=='Urdu'].index,inplace=True)

In [ ]:
df[df['lang']=="Urdu"]

,Text,labels,lang


In [ ]:
df[df['lang']=="Hindi"]

,Text,labels,lang
19680,आई एम बग फन आफ सरय,Non-sarcastic,Hindi
20099,द बसट द बसट द बसट ऑफ बसट,Non-sarcastic,Hindi
23828,पटट परक सपरसटरस रजनकत,Non-sarcastic,Hindi


In [ ]:
df.drop(df[df['lang']=='Hindi'].index,inplace=True)

In [ ]:
df[df['lang']=='Hindi']

,Text,labels,lang


In [ ]:
df[df['lang']=="Western Frisian"]

,Text,labels,lang
27297,kole gandule irruken mavane kolleme vidde mate...,Sarcastic,Western Frisian


In [ ]:
df.loc[27297,'lang']='English'
df.loc[27297]


Text      kole gandule irruken mavane kolleme vidde mate...
labels                                            Sarcastic
lang                                                English
Name: 27297, dtype: object

In [ ]:
df[df['lang']=='Sindhi']

,Text,labels,lang
26431,صديق سوي بيض زياده جبنه شطة,Non-sarcastic,Sindhi


In [ ]:
df.drop(26431,inplace=True)

In [ ]:
df[df['lang']=='Gujarati']
df=df.drop(21905)
df[df['lang']=='Gujarati']

,Text,labels,lang


In [ ]:
df[df['lang']=='Bhojpuri']
df=df.drop([19096,20850])
df[df['lang']=='Bhojpuri']


,Text,labels,lang


In [ ]:
df[df['lang']=='Vietnamese']
df=df.drop(23000)
df[df['lang']=='Vietnamese']


,Text,labels,lang
18536,kuch nayi story nhi tum logo ke paas only biopcs,Non-sarcastic,Vietnamese


In [ ]:
df[df['lang']=='Greek']
df=df.drop(14789)
df[df['lang']=='Greek']

,Text,labels,lang


In [ ]:
df[df['lang']=='Northern Sotho']
df.loc[df['lang']=="Northern Sotho",'lang']="English"
df[df['lang']=='Northern Sotho']

,Text,labels,lang


In [ ]:
df[df['lang']=='Zhuang']
df.loc[df['lang']=="Zhuang","lang"]="English"
df[df['lang']=='Zhuang']


,Text,labels,lang


In [ ]:
df[df['lang']=='Morisyen']
df.drop(25036,inplace=True)

In [ ]:
df[df['lang']=="Morisyen"]
df.loc[df['lang']=="Morisyen","lang"]="English"
df[df['lang']=='Morisyen']

,Text,labels,lang


In [ ]:
df[df['lang']=='Southern Sotho']
df=df.drop([7082,16060,19699])
df[df['lang']=='Southern Sotho']

,Text,labels,lang
9603,jeyikuromo illa thokuromo modhalla sanda seiyanum,Sarcastic,Southern Sotho
14629,unmai sonnal kasakkutho mohan avarkalin patham...,Non-sarcastic,Southern Sotho
17389,devidiya padam punda aenda love panna ennada t...,Non-sarcastic,Southern Sotho
25647,vijay sethupathya pidichavan like ah thatuu,Non-sarcastic,Southern Sotho
28418,thalapathy kku evlo massss irukka pootho thala...,Sarcastic,Southern Sotho


In [ ]:
df.loc[df['lang']=="Southern Sotho",'lang']="English"
df[df['lang']=='Southern Sotho']

,Text,labels,lang


In [ ]:
df[df['lang']=="Tatar"]
df=df.drop([13127,18654])
df.loc[df['lang']=="Tatar",'lang']="English"
df[df['lang']=='Tatar']

,Text,labels,lang


In [ ]:
df[df['lang']=="Maltese"]
df=df.drop([6325,22665])
df.loc[df['lang']=="Maltese",'lang']="English"
df[df['lang']=="Maltese"]

,Text,labels,lang


In [ ]:
df[df['lang']=="Russian"]
df=df.drop([5942,12492,16652])
df[df['lang']=="Russian"]

,Text,labels,lang


In [ ]:
df[df['lang']=="Catalan"]
df.loc[df['lang']=="Catalan",'lang']="English"
df[df['lang']=="Catalan"]


,Text,labels,lang


In [ ]:
df[df['lang']=="Inupiaq"]
df.loc[df['lang']=="Inupiaq",'lang']="English"
df[df['lang']=="Inupiaq"]


,Text,labels,lang


In [ ]:
df[df['lang']=="Kannada"]
df.drop(df[df['lang']=='Kannada'].index,inplace=True)
df[df['lang']=="Kannada"]

,Text,labels,lang


In [ ]:
df[df['lang']=="Interlingua"]
df.drop(12759,inplace=True)
df.loc[df['lang']=="Interlingua",'lang']="English"
df[df['lang']=="Interlingua"]

,Text,labels,lang


In [ ]:
df[df['lang']=="unknown"].tail(20)

df.loc[[27768,26171],'lang']='Tamil'
df[df['lang']=="unknown"].tail(20)

,Text,labels,lang
27333,hair style nalla ella pa,Sarcastic,unknown
27347,കരള mohanlal and suriya fans അട മകകള like,Non-sarcastic,unknown
27360,мaѕѕ waιтιng ғor ԹҽԵԵɑ,Non-sarcastic,unknown
27387,thalaivar oda style ah thanni dan sweet sapid...,Non-sarcastic,unknown
27532,naan ketkka virumbuvadhu orey oru kelvi oru sc...,Non-sarcastic,unknown
27537,iyoo therikuthuenna style ah irukaru ya thalai...,Non-sarcastic,unknown
27831,title kagave movie pakka poren,Non-sarcastic,unknown
27845,daii un fans ah ipadii kathara vachuttiye da d...,Sarcastic,unknown
28245,thala poga poga style agikitte poreenga summa...,Sarcastic,unknown
28262,vikram hair stylepidichavanga like podunga pa ...,Non-sarcastic,unknown


In [ ]:
df=df.drop(4)


In [ ]:
df[df['lang']=="unknown"].tail(50)
df.drop([22505,22986],inplace=True)
df.loc[26035,'lang']="Tamil"

In [ ]:
df[df['lang']=="unknown"].tail(60)
df.drop([21458])
df.loc[[19332,21600],'lang']="Tamil"

In [ ]:
df[df['lang']=="unknown"].tail(70)
df.loc[[18577,663],'lang']='Tamil'
x=df[df['lang']=="unknown"]
print(x)

                                                    Text         labels  \
326    enna da telugu padam maariye eduthu vachi iruk...      Sarcastic   
338    ambedkar jaadhi venam madham venanu sonnaru na...  Non-sarcastic   
378    makka lai kanyakumari vanthu parunga la inka t...  Non-sarcastic   
999    uyir kurudhiyil urudhiyai serada dhisai engilu...  Non-sarcastic   
1063   thalaivaaaaaaaa pakka mass invincible style sa...  Non-sarcastic   
...                                                  ...            ...   
28934                   title la teaser ku r ah kanum pa  Non-sarcastic   
28960  ottu mothama sethuruchi intha trailer nu solra...  Non-sarcastic   
29058  antha kaalathula kaasu irukavan aanda parambar...  Non-sarcastic   
29120  yaarulam class la mam class edukum pothu seric...  Non-sarcastic   
29144                 malayalam ൽ mohanlal tamil ൽ surya  Non-sarcastic   

          lang  
326    unknown  
338    unknown  
378    unknown  
999    unknown  
1063   unknown

In [ ]:
df[df['lang']=="unknown"].tail(10)
df=df.drop(29144)


In [ ]:
df[df['lang']=="unknown"].tail(20)
df=df.drop([27347,27360])


In [ ]:
df[df['lang']=="unknown"].head(20)
df.loc[[3251,3214,4877],'lang']="Tamil"
df[df['lang']=="unknown"].head(40)
df=df.drop([3611,4013,5023,5840])
df[df['lang']=="unknown"].head(40)

,Text,labels,lang
326,enna da telugu padam maariye eduthu vachi iruk...,Sarcastic,unknown
338,ambedkar jaadhi venam madham venanu sonnaru na...,Non-sarcastic,unknown
378,makka lai kanyakumari vanthu parunga la inka t...,Non-sarcastic,unknown
999,uyir kurudhiyil urudhiyai serada dhisai engilu...,Non-sarcastic,unknown
1063,thalaivaaaaaaaa pakka mass invincible style sa...,Non-sarcastic,unknown
1110,t h a l a,Non-sarcastic,unknown
1164,ok sir enaku ore sila kelvigal mattum thaa iru...,Sarcastic,unknown
1174,talaa ajith sir wt a style,Non-sarcastic,unknown
1767,thevdiya paiya director nu sonna unakku kovam ...,Sarcastic,unknown
1970,script romba nalla iruku conform block buster ...,Non-sarcastic,unknown


In [ ]:
df[df['lang']=="unknown"].head(50)
df=df.drop(7301)
df.loc[[7571],'lang']="Tamil"

In [ ]:
df[df['lang']=="unknown"].tail(60)
df=df.drop([21458])


In [ ]:
df[df['lang']=="unknown"].tail(80)
df=df.drop([17147])
df.loc[[17139],"lang"]='Tamil'
df[df['lang']=="unknown"].tail(90)

,Text,labels,lang
15237,sorry thalaivaa yesterday bhairava pakke poir...,Sarcastic,unknown
15492,thalaiva unge style ku nanga adimai,Non-sarcastic,unknown
15496,yenda neenga mattum unga paranjith movie la va...,Non-sarcastic,unknown
15766,super padam sirnalla msg solli irukingagovt ve...,Non-sarcastic,unknown
16093,izu vijayda traileraaha irunda 30 millions aah...,Sarcastic,unknown
...,...,...,...
28839,pakkathan ah pora intha kaali oda aatatha ra...,Sarcastic,unknown
28934,title la teaser ku r ah kanum pa,Non-sarcastic,unknown
28960,ottu mothama sethuruchi intha trailer nu solra...,Non-sarcastic,unknown
29058,antha kaalathula kaasu irukavan aanda parambar...,Non-sarcastic,unknown


In [ ]:
df[df['lang']=="unknown"].tail(100)
df=df.drop([14037,14232])
df[df['lang']=="unknown"].tail(100)

,Text,labels,lang
13719,thalaiva semma style pakka mass,Non-sarcastic,unknown
13884,t h a l a,Non-sarcastic,unknown
14155,yepomay class la sirichi thittu vaanguravanga ...,Non-sarcastic,unknown
14349,varaar varaar varaar nadippin nayakan thirumbi...,Non-sarcastic,unknown
14580,ota tharamaana sambavam ini oru beep group kel...,Non-sarcastic,unknown
...,...,...,...
28839,pakkathan ah pora intha kaali oda aatatha ra...,Sarcastic,unknown
28934,title la teaser ku r ah kanum pa,Non-sarcastic,unknown
28960,ottu mothama sethuruchi intha trailer nu solra...,Non-sarcastic,unknown
29058,antha kaalathula kaasu irukavan aanda parambar...,Non-sarcastic,unknown


In [ ]:
df[df['lang']=="unknown"].tail(120)
df.loc[[11207,11260],'lang']="Tamil"
df=df.drop(11711)
df[df['lang']=="unknown"].tail(120)

,Text,labels,lang
11057,thala tharisanam miga viraivil மரண மாஸ் my dea...,Sarcastic,unknown
11063,dei arivu kettavana naan 5 class da en venru,Sarcastic,unknown
11129,classmass vartha varala sema wo wo wo,Non-sarcastic,unknown
11716,entha gethu mass yarukuda varum thalaivar...,Sarcastic,unknown
11742,thalaivaaaaaaaa pakka mass invincible style sa...,Non-sarcastic,unknown
...,...,...,...
28839,pakkathan ah pora intha kaali oda aatatha ra...,Sarcastic,unknown
28934,title la teaser ku r ah kanum pa,Non-sarcastic,unknown
28960,ottu mothama sethuruchi intha trailer nu solra...,Non-sarcastic,unknown
29058,antha kaalathula kaasu irukavan aanda parambar...,Non-sarcastic,unknown


In [ ]:
df.loc[df['lang']=='unknown','lang']='English'


In [ ]:
df.loc[df['lang']=="Nyanja"].tail(150)
df=df.drop([17910])
df.loc[df['lang']=="Nyanja"].tail(155)

,Text,labels,lang
17352,athane vels produce pandrange na varun ilameleyaa,Non-sarcastic,Nyanja
17473,kottha inime intha sc naikal yellaru m padatha...,Non-sarcastic,Nyanja
17543,sourashtra sarpaga naanum atharikirean kothra...,Non-sarcastic,Nyanja
17591,ithula jathiya thandi ponna pathi kathai ithu ...,Non-sarcastic,Nyanja
17713,trailer release aaki 16 days aakuthu indai var...,Sarcastic,Nyanja
...,...,...,...
28938,vitrakudathu 1 st place athula record pannuvom,Sarcastic,Nyanja
29095,trailer muluka sirika vechitingale adhi anna,Non-sarcastic,Nyanja
29127,ovoru theruvilayum ovoru naium dog gethu than ...,Non-sarcastic,Nyanja
29337,morattu thanama irruku ya nanu ippola irrunthu...,Non-sarcastic,Nyanja


In [ ]:
df.loc[df['lang']=="Nyanja","lang"]="English"
df[df['lang']=="Nyanja"]

,Text,labels,lang


In [ ]:
df.loc[df['lang']=="Malay","lang"]="English"
df[df['lang']=="Malay"]

,Text,labels,lang


In [ ]:
df.loc[df['lang']=="Sanskrit","lang"]="English"
df[df['lang']=="Sanskrit"]

,Text,labels,lang


In [ ]:
df

,Text,labels,lang
0,அர்யவுக்கு ஒரு நல்ல வாய்ப்பு சிங்கம் சூரியா அ...,Non-sarcastic,Tamil
1,பள்ளியோ கல்லூரியோ படித்துக்கொண்டிருக்கும்போது ...,Non-sarcastic,Tamil
2,தல தல தல தல தல தல தல தல தல தல தல தல தல தல தல த...,Non-sarcastic,Tamil
3,all the best to the teamsuper ah na oru kadhai...,Non-sarcastic,English
5,apadilam nadakathu nadakavum koodathu mass dia...,Non-sarcastic,English
...,...,...,...
29565,ajth sar mass vasanam ner konda baarvai mass,Non-sarcastic,English
29566,adei anga enpt trailer mukkitu iruku innum,Sarcastic,English
29567,kv anandh sir sentiment ayan maatran kavan ma...,Non-sarcastic,English
29568,ஒவ்வொருத்தருக்கும் ஒவ்வொரு பைத்தியம் நமக்கு யு...,Non-sarcastic,Tamil


In [ ]:
df[df['lang']=="Malayalam"]
df.drop(df[df['lang']=='Malayalam'].index,inplace=True)
df[df['lang']=="Malayalam"]

,Text,labels,lang


In [ ]:
df[df['lang']=="Telugu"]
df.drop(df[df['lang']=='Telugu'].index,inplace=True)
df[df['lang']=="Telugu"]

,Text,labels,lang


In [ ]:
df[df['lang']=="Spanish"]
df=df.drop([12636,15677,20532,21487])
df[df['lang']=="Spanish"]

,Text,labels,lang
94,luv u surya soooooo muchhhhhh,Non-sarcastic,Spanish
4252,hay ithu 96 yara yamathureenga,Non-sarcastic,Spanish
10572,pudikalana koothiya mooditu pongalendacomments...,Sarcastic,Spanish
12658,bgm verla levelu1 thalaivan eppovum masss,Non-sarcastic,Spanish
14483,telugu lo pawan kalyan tamil lo surya,Sarcastic,Spanish


In [ ]:
df.loc[df['lang']=="Spanish","lang"]="English"
df.loc[df['lang']=="Spanish"]

,Text,labels,lang


In [ ]:
df[df['lang']=="German"]
df=df.drop([5001,12345,12680,18102,18521,25648])
df.loc[df['lang']=="German","lang"]="English"
df[df['lang']=="German"]


,Text,labels,lang


In [ ]:
df[df['lang']=="Polish"]
df=df.drop([6206,10383,24554])
df.loc[df['lang']=="Polish","lang"]="English"
df[df['lang']=="Polish"]

,Text,labels,lang


In [ ]:
df[df['lang']=="Serbian"]
df=df.drop([961])
df[df['lang']=="Serbian"]

,Text,labels,lang


In [ ]:
df.loc[df['lang']=="Irish","lang"]="English"
df[df['lang']=="Irish"]


,Text,labels,lang


In [ ]:
df[df['lang']=="French"]
df=df.drop([5406,5506,5910,7293,10402,13317,19683,21517,22959,21535,22519,22959])


In [ ]:
df.loc[df['lang']=="French","lang"]="English"
df[df['lang']=="French"]


,Text,labels,lang


In [ ]:
df[df['lang']=="Afrikaans"]
df=df.drop([496,2165,3829,9279,12571,28502])
df.loc[df['lang']=="Afrikaans","lang"]="English"
df[df['lang']=="Afrikaans"]


,Text,labels,lang


In [ ]:
df[df['lang']=="Slovenian"]
df=df.drop([14196,19398])
df.loc[df['lang']=="Slovenian","lang"]="English"
df[df['lang']=="Slovenian"]

,Text,labels,lang


In [ ]:
df[df['lang']=="Slovak"]


,Text,labels,lang
179,ennada sk paithiyakara maari dialogue pesama i...,Non-sarcastic,Slovak
523,sk dhillu than abhey deol kudu nadikarathukku,Non-sarcastic,Slovak
2347,dislike pottuvan sk fan ah tha eruppan echa sh...,Sarcastic,Slovak
4284,bole to ak dam jhakash,Non-sarcastic,Slovak
4980,sk fans saapra valthukkal pa,Sarcastic,Slovak
...,...,...,...
27068,dhanush oda character puthu sa erukay mass ta,Non-sarcastic,Slovak
27518,marvel aavuthu mairavathu sk universe daw,Sarcastic,Slovak
27929,sk fans saarpa valthukkal dhanush anna,Non-sarcastic,Slovak
28448,sk fans like annan uyir fans comment,Non-sarcastic,Slovak


In [ ]:
df[df['lang']=="unknown"].head(100)
df.loc[7339,'lang']='Tamil'
df.loc[7339]



Text      aaga தளபதி fans ku oru kuthu தல fans ku oru ku...
labels                                            Sarcastic
lang                                                  Tamil
Name: 7339, dtype: object

In [ ]:
df[df['lang']=="Slovak"]
df=df.drop([12023])
df.loc[df['lang']=="Slovak","lang"]="English"
df[df['lang']=="Slovak"]



,Text,labels,lang


In [ ]:
df[df['lang']=="Khasi"]
df.loc[df['lang']=="Khasi","lang"]="English"
df[df['lang']=="Khasi"]
df[df['lang']=="Quechua"]
df=df.drop([28835,28148])
df.loc[df['lang']=="Quechua","lang"]="English"
df[df['lang']=="Quechua"]
df[df['lang']=="Manx"]


,Text,labels,lang
27,thalabathy ku pidicha nadigar thala enakum t...,Non-sarcastic,Manx
231,pandey thalaya vey ethirthu pesuriya goyala th...,Sarcastic,Manx
289,thalaivaaaaaaa str,Non-sarcastic,Manx
935,dislike panavangalaukula nalla vairu earithu p...,Sarcastic,Manx
948,ponunga ku mariyadai thalai ku visvasam life...,Non-sarcastic,Manx
...,...,...,...
28309,thalaiva podu thakida thakida like podu ga tha...,Non-sarcastic,Manx
29021,nee vaaaa thalaivaaa bairavaa,Sarcastic,Manx
29090,thala ungala minja ala eillahats off,Non-sarcastic,Manx
29421,dai pandey thala ya vey yethirthu pesuriya goy...,Sarcastic,Manx


In [ ]:
df[df['lang']=="Manx"]
df.loc[df['lang']=="Manx","lang"]="English"
df[df['lang']=="Manx"]
df[df['lang']=="Azerbaijani"]
df.loc[df['lang']=="Azerbaijani","lang"]="English"
df[df['lang']=="Azerbaijani"]
df[df['lang']=="Turkish"]
df=df.drop([6370,9167,19628])
df.loc[df['lang']=="Turkish","lang"]="English"
df[df['lang']=="Turkish"]



,Text,labels,lang


In [ ]:
df[df['lang']=='Manx']

,Text,labels,lang


In [ ]:
df[df['lang']=="Swahili"]
df=df.drop(28070)
df.loc[df['lang']=="Swahili","lang"]="English"
df[df['lang']=="Swahili"]

,Text,labels,lang


In [ ]:
df[df['lang']=='Kinyarwanda']
df=df.drop([678,1066,1228,1866])
df.loc[df['lang']=='Kinyarwanda','lang']='English'
df[df['lang']=='Kinyarwanda']

,Text,labels,lang


In [ ]:
df[df['lang']=='Oromo']
df.loc[df['lang']=='Oromo','lang']='English'
df[df['lang']=='Oromo']
df[df['lang']=='Klingon']
df.loc[df['lang']=='Klingon','lang']='English'
df[df['lang']=='Klingon']
df[df['lang']=='Xhosa']
df.loc[df['lang']=='Xhosa','lang']='English'
df[df['lang']=='Xhosa']
df[df['lang']=='Indonesian']
df=df.drop([29342,29444])
df.loc[df['lang']=='Indonesian','lang']='English'
df[df['lang']=='Indonesian']



,Text,labels,lang


In [ ]:
df[df['lang']=='Estonian']
df=df.drop([72,505])
df.loc[df['lang']=='Estonian','lang']='English'
df[df['lang']=='Estonian']

,Text,labels,lang


In [ ]:
df[df['lang']=='Finnish']
df.loc[471,'lang']='Tamil'
df.loc[df['lang']=='Finnish','lang']="English"
df[df['lang']=='Finnish']
df[df['lang']=='Zulu']
df.loc[df['lang']=='Zulu','lang']='English'
df[df['lang']=='Zulu']
df[df['lang']=='Scottish Gaelic']
df.loc[df['lang']=='Scottish Gaelic','lang']='English'
df[df['lang']=='Scottish Gaelic']
df[df['lang']=='Galician']
df=df.drop([147,595,28504,28694,28803])
df.loc[df['lang']=='Galician','lang']='English'
df[df['lang']=='Galician']



,Text,labels,lang


In [ ]:
df[df['lang']=='zzp']
df.loc[df['lang']=='zzp','lang']='English'
df[df['lang']=='zzp']
df[df['lang']=='Rundi']
df.loc[df['lang']=='Rundi','lang']='English'
df[df['lang']=='Rundi']
df[df['lang']=='Lithuanian']
df=df.drop(27976)
df.loc[df['lang']=='Lithuanian','lang']='English'
df[df['lang']=='Lithuanian']

,Text,labels,lang


In [ ]:
df[df['lang']=='Javanese']
df.loc[df['lang']=='Javanese','lang']='English'
df[df['lang']=='Javanese']
df[df['lang']=='Scots']
df.loc[df['lang']=='Scots','lang']='English'
df[df['lang']=='Scots']
df[df['lang']=='Wolof']
df.loc[df['lang']=='Wolof','lang']='English'
df[df['lang']=='Wolof']
df[df['lang']=='Tagalog']
df.loc[df['lang']=='Tagalog','lang']='English'
df[df['lang']=='Tagalog']
df[df['lang']=='Afar']
df.loc[df['lang']=='Afar','lang']='English'
df[df['lang']=='Afar']
df[df['lang']=='Venda'].head(50)
df.loc[df['lang']=='Venda','lang']='English'
df[df['lang']=='Venda']
df[df['lang']=='Shona']
df.loc[df['lang']=='Shona','lang']='English'
df[df['lang']=='Shona']
df[df['lang']=='Fijian']
df.loc[df['lang']=='Fijian','lang']='English'
df[df['lang']=='Fijian']
df[df['lang']=='Aymara']
df.loc[df['lang']=='Aymara','lang']='English'
df[df['lang']=='Aymara']
df[df['lang']=='Somali']
df.loc[df['lang']=='Somali','lang']='English'
df[df['lang']=='Somali']
df[df['lang']=='Esperanto']
df.loc[19566,'lang']='Tamil'
df.loc[df['lang']=='Esperanto','lang']='English'
df[df['lang']=='Esperanto']
df[df['lang']=='Uzbek']
df.loc[df['lang']=='Uzbek','lang']='English'
df[df['lang']=='Uzbek']
df[df['lang']=='Interlingue']
df.loc[df['lang']=='Interlingue','lang']='English'
df[df['lang']=='Interlingue']

,Text,labels,lang


In [ ]:
l=df['lang'].unique()
for x in l:
  print(x)

Tamil
English
Danish
Norwegian Nynorsk
Corsican
Breton
Igbo
Kalaallisut
Ganda
Malagasy
Tswana
Latin
Hausa
Yoruba
Seselwa Creole French
Icelandic
Swedish
Albanian
Turkmen
Faroese
Arabic
Nauru
Welsh
Hungarian
Lingala
Czech
Samoan
Romanian
Norwegian
Bosnian
Basque
Romansh
Swati
Tsonga
Māori
Bislama
Akan
Croatian
Sundanese
Dutch
Luxembourgish
Tongan
Hmong
Portuguese
Waray
Occitan
Cebuano
Italian
Hawaiian
Guarani
Sango
Volapük
Haitian Creole
Latvian
Vietnamese


In [ ]:
df[df['lang']=='Corsican']
df.loc[df['lang']=='Corsican','lang']='English'
df[df['lang']=='Corsican']

,Text,labels,lang


In [ ]:
df[df['lang']=='Breton']
df.loc[df['lang']=='Breton','lang']='English'
df[df['lang']=='Breton']
df[df['lang']=='Igbo']
df.loc[df['lang']=='Igbo','lang']='English'
df[df['lang']=='Igbo']
df[df['lang']=='Kalaallisut']
df.loc[df['lang']=='Kalaallisut','lang']='English'
df[df['lang']=='Kalaallisut']
df[df['lang']=='Ganda']
df=df.drop(23169)
df.loc[df['lang']=='Ganda','lang']='English'
df[df['lang']=='Ganda']

,Text,labels,lang


In [ ]:
df[df['lang']=='Albanian']
df.loc[df['lang']=='Albanian','lang']='English'
df[df['lang']=='Albanian']
df[df['lang']=='Turkmen']
df.loc[df['lang']=='Turkmen','lang']='English'
df[df['lang']=='Turkmen']
df[df['lang']=='Faroese'].head(60)
df.loc[12525,'lang']='Tamil'
df.loc[df['lang']=='Faroese','lang']='English'
df[df['lang']=='Faroese']
df[df['lang']=='Arabic']
df = df.drop(df[df['lang'] == 'Arabic'].index)




In [ ]:
df[df['lang']=="Danish"]
df.loc[df['lang']=="Danish","lang"]="English"
df[df['lang']=="Danish"]
df[df['lang']=='Norwegian Nynorsk']
df=df.drop([29327])
df.loc[df['lang']=="Norwegian Nynorsk","lang"]="English"
df[df['lang']=='Norwegian Nynorsk']


,Text,labels,lang


In [ ]:
df[df['lang']=='Arabic']
df[df['lang']=='Welsh']
df.loc[df['lang']=='Welsh','lang']='English'
df[df['lang']=='Welsh']
df[df['lang']=='Nauru']
df.loc[df['lang']=='Nauru','lang']='English'
df[df['lang']=='Nauru']
df[df['lang']=='Hungarian']
df.loc[df['lang']=='Hungarian','lang']='English'
df[df['lang']=='Hungarian']
df[df['lang']=='Lingala']
df.loc[df['lang']=='Lingala','lang']='English'
df[df['lang']=='Lingala']
df[df['lang']=='Czech']
df.loc[df['lang']=='Czech','lang']='English'
df[df['lang']=='Czech']
df[df['lang']=='Samoan']
df.loc[df['lang']=='Samoan','lang']='English'
df[df['lang']=='Samoan']

,Text,labels,lang


In [ ]:
df[df['lang']=='Romanian']
df.loc[df['lang']=='Romanian','lang']='English'
df[df['lang']=='Romanian']
df[df['lang']=='Norwegian']
df.loc[df['lang']=='Norwegian','lang']='English'
df[df['lang']=='Norwegian']
df[df['lang']=='Bosnian']
df.loc[df['lang']=='Bosnian','lang']='English'
df[df['lang']=='Bosnian']
df[df['lang']=='Basque']
df.loc[df['lang']=='Basque','lang']='English'
df[df['lang']=='Basque']
df[df['lang']=='Romansh']
df.loc[df['lang']=='Romansh','lang']='English'
df[df['lang']=='Romansh']
df[df['lang']=='Swati']
df.loc[df['lang']=='Swati','lang']='English'
df[df['lang']=='Swati']
df[df['lang']=='Tsonga']
df.loc[df['lang']=="Tsonga",'lang']="English"
df[df['lang']=="Tsonga"]
df[df['lang']=="Māori"]
df.loc[df['lang']=="Māori",'lang']="English"
df[df['lang']=="Māori"]
df[df['lang']=="Bislama"]
df.loc[df['lang']=="Bislama",'lang']="English"
df[df['lang']=="Bislama"]
df[df['lang']=='Akan']
df.loc[df['lang']=='Akan','lang']='English'
df[df['lang']=='Akan']
df[df['lang']=='Croatian']
df.loc[df['lang']=='Croatian','lang']='English'
df[df['lang']=='Croatian']
df[df['lang']=='Sundanese']
df.loc[df['lang']=='Sundanese','lang']='English'
df[df['lang']=='Sundanese']
df[df['lang']=='Dutch']
df.loc[df['lang']=='Dutch','lang']='English'
df[df['lang']=='Dutch']

,Text,labels,lang


In [ ]:
df[df['lang']=='Luxembourgish']
df.loc[df['lang']=='Luxembourgish','lang']='English'
df[df['lang']=='Luxembourgish']
df[df['lang']=='Tongan']
df.loc[df['lang']=='Tongan','lang']='English'
df[df['lang']=='Tongan']
df[df['lang']=='Hmong']
df.loc[df['lang']=='Hmong','lang']='English'
df[df['lang']=='Hmong']
df[df['lang']=='Portuguese']
df=df.drop(20490)
df.loc[df['lang']=='Portuguese','lang']='English'
df[df['lang']=='Portuguese']

,Text,labels,lang


In [ ]:
df[df['lang']=='Waray']
df.loc[df['lang']=='Waray','lang']='English'
df[df['lang']=='Waray']
df[df['lang']=='Occitan']
df.loc[df['lang']=='Occitan','lang']='English'
df[df['lang']=='Occitan']
df[df['lang']=='Cebuano']
df.loc[df['lang']=='Cebuano','lang']='English'
df[df['lang']=='Cebuano']
df[df['lang']=='Italian']
df.loc[df['lang']=='Italian','lang']='English'
df[df['lang']=='Italian']
df[df['lang']=='Hawaiian']
df.loc[df['lang']=='Hawaiian','lang']='English'
df[df['lang']=='Hawaiian']
df[df['lang']=='Guarani']
df.loc[df['lang']=='Guarani','lang']='English'
df[df['lang']=='Guarani']
df[df['lang']=='Sango']
df.loc[df['lang']=='Sango','lang']='English'
df[df['lang']=='Sango']
df[df['lang']=='Volapük']
df.loc[df['lang']=='Volapük','lang']='English'
df[df['lang']=='Volapük']
df[df['lang']=='Haitian Creole']
df.loc[df['lang']=='Haitian Creole','lang']='English'
df[df['lang']=='Haitian Creole']
df[df['lang']=='Latvian']
df.loc[df['lang']=='Latvian','lang']='English'
df[df['lang']=='Latvian']

,Text,labels,lang


In [ ]:
df[df['lang']=='Vietnamese']
df.loc[df['lang']=='Vietnamese','lang']='English'
df[df['lang']=='Vietnamese']

,Text,labels,lang


In [ ]:
l=df['lang'].unique()
for x in l:
  print(x)

Tamil
English
Malagasy
Tswana
Latin
Hausa
Yoruba
Seselwa Creole French
Icelandic
Swedish


In [ ]:
df[df['lang']=='Malagasy']
df.loc[7234,'lang']='Tamil'
df.loc[df['lang']=='Malagasy','lang']='English'
df[df['lang']=='Malagasy']
df[df['lang']=='Tswana']
df.loc[df['lang']=='Tswana','lang']='English'
df[df['lang']=='Tswana']
df[df['lang']=='Latin']
df.loc[df['lang']=='Latin','lang']='English'
df[df['lang']=='Latin']
df[df['lang']=='Hausa']
df.loc[df['lang']=='Hausa','lang']='English'
df[df['lang']=='Hausa']

df[df['lang']=='Seselwa Creole French']
df.loc[df['lang']=='Seselwa Creole French','lang']='English'
df[df['lang']=='Seselwa Creole French']
df[df['lang']=='Icelandic']
df.loc[df['lang']=='Icelandic','lang']='English'
df[df['lang']=='Icelandic']
df[df['lang']=='Swedish']
df.loc[df['lang']=='Swedish','lang']='English'
df[df['lang']=='Swedish']

,Text,labels,lang


In [ ]:
df[df['lang']=='Yoruba']
df.loc[df['lang']=='Yoruba','lang']='English'
df[df['lang']=='Yoruba']

,Text,labels,lang


In [ ]:
l=df['lang'].unique()
for x in l:
  print(x)

Tamil
English


In [ ]:
stop_words = [
    "எந்த", "என்று", "எனவே", "ஏனெனில்", "அங்கு", "அதில்", "அவர்", "அவ்வாறு", "அல்லது", "அவர்கள்",
    "அவை", "ஆக", "இங்கு", "இதை", "இதைத்", "இது", "இதில்", "இந்த", "இந்தப்", "இதற்கு", "இதனால்",
    "இவைகள்", "இவர்கள்", "இவை", "இவை", "உள்ள", "என", "ஏன்", "எப்பொழுது", "எப்போது", "எது", "எது",
    "ஒரு", "ஒன்றின்", "ஒரே", "உள்ள", "மற்றும்", "மிக", "மிகவும்", "வெளியே", "என்பதை", "என்பது",
    "என்கிற", "என்று", "சில", "சிலரின்", "மற்றும்", "சிறிய", "சிறிது", "சிறு", "தன்", "தனது",
    "இருக்கும்", "பின்", "பின்னர்", "மீண்டும்", "ஆனால்", "அதனால்", "போல்", "மேலும்", "மேலேயே",
    "அதற்கு", "அதை", "என்ன", "என்றால்", "அவரது", "உங்கள்", "நான்", "எனக்கு", "எனது", "நீங்கள்"
]



In [ ]:
import spacy
from spacy.lang.ta import Tamil

# Create a custom Tamil stop words list
tamil_stop_words = set(stop_words)

# Load a Tamil language model
nlp = Tamil()

# Extend the spaCy stop word list with custom Tamil stop words
for word in tamil_stop_words:
    nlp.vocab[word].is_stop = True

print(tamil_stop_words)


{'மீண்டும்', 'ஆனால்', 'சிறிய', 'அதில்', 'இங்கு', 'என்பது', 'எனவே', 'போல்', 'அவ்வாறு', 'உள்ள', 'எனது', 'என்று', 'மிக', 'ஏன்', 'மேலேயே', 'ஏனெனில்', 'இவை', 'வெளியே', 'இவைகள்', 'மேலும்', 'என்றால்', 'அவர்', 'இதைத்', 'இது', 'ஒரு', 'மற்றும்', 'பின்', 'ஒன்றின்', 'அதை', 'அவரது', 'என', 'இந்தப்', 'எப்போது', 'சிறிது', 'தன்', 'பின்னர்', 'இந்த', 'அதற்கு', 'நீங்கள்', 'இருக்கும்', 'இதை', 'இதில்', 'எந்த', 'எப்பொழுது', 'அதனால்', 'சிலரின்', 'இதனால்', 'என்கிற', 'ஆக', 'எனக்கு', 'என்பதை', 'ஒரே', 'சிறு', 'என்ன', 'அங்கு', 'அல்லது', 'அவர்கள்', 'சில', 'தனது', 'இதற்கு', 'உங்கள்', 'மிகவும்', 'இவர்கள்', 'எது', 'அவை', 'நான்'}


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Get the list of Tamil stopwords
stop_words = tamil_stop_words

def remove_stopwords(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Reconstruct the text
    return ' '.join(filtered_tokens)

# Apply the function to your DataFrame
df['Text'] = df['Text'].apply(remove_stopwords)

# Display the cleaned DataFrame
df


,Text,labels,lang
0,அர்யவுக்கு நல்ல வாய்ப்பு சிங்கம் சூரியா அண்ணவோ...,Non-sarcastic,Tamil
1,பள்ளியோ கல்லூரியோ படித்துக்கொண்டிருக்கும்போது ...,Non-sarcastic,Tamil
2,தல தல தல தல தல தல தல தல தல தல தல தல தல தல தல த...,Non-sarcastic,Tamil
3,all the best to the teamsuper ah na oru kadhai...,Non-sarcastic,English
5,apadilam nadakathu nadakavum koodathu mass dia...,Non-sarcastic,English
...,...,...,...
29565,ajth sar mass vasanam ner konda baarvai mass,Non-sarcastic,English
29566,adei anga enpt trailer mukkitu iruku innum,Sarcastic,English
29567,kv anandh sir sentiment ayan maatran kavan maa...,Non-sarcastic,English
29568,ஒவ்வொருத்தருக்கும் ஒவ்வொரு பைத்தியம் நமக்கு யு...,Non-sarcastic,Tamil


In [ ]:
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list.extend(['ena','unaku','per','irukanga','panna','yarum','mattum','ivan','ada','pesa','unakku','k','sari','idhu','vida','vittu','enga','yen','ithu','poda','dey','irundhu','ya','la', 'u','r','s','bro','da','dei','dai','nu','ah','nee','ni','illa','un','ok','na','pls','ur','unga'])
stopwords_list.extend(['🥰','indha','antha','vera','iruka','pola','innum','avan','summa','ellam','thaan','romba','❤️','ana','ama','apdi','ithula','po','evlo','eruku','irukum','nama','enna','va','hi','h','ku','iruku','naa','va','oru','athu','avanga','neenga','tha','en','di','dhan','ne','ella','intha'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stopwords_list]
    # Reconstruct the text
    return ' '.join(filtered_tokens)

# Apply the function to your DataFrame
df['Text'] = df['Text'].apply(remove_stopwords)

# Display the cleaned DataFrame
df

,Text,labels,lang
0,அர்யவுக்கு நல்ல வாய்ப்பு சிங்கம் சூரியா அண்ணவோ...,Non-sarcastic,Tamil
1,பள்ளியோ கல்லூரியோ படித்துக்கொண்டிருக்கும்போது ...,Non-sarcastic,Tamil
2,தல தல தல தல தல தல தல தல தல தல தல தல தல தல தல த...,Non-sarcastic,Tamil
3,best teamsuper kadhai karuippa irukura ponnuga...,Non-sarcastic,English
5,apadilam nadakathu nadakavum koodathu mass dia...,Non-sarcastic,English
...,...,...,...
29565,ajth sar mass vasanam ner konda baarvai mass,Non-sarcastic,English
29566,adei anga enpt trailer mukkitu,Sarcastic,English
29567,kv anandh sir sentiment ayan maatran kavan maa...,Non-sarcastic,English
29568,ஒவ்வொருத்தருக்கும் ஒவ்வொரு பைத்தியம் நமக்கு யு...,Non-sarcastic,Tamil


In [ ]:
df['labels'] = df['labels'].map({'Sarcastic': 1, 'Non-sarcastic': 0})
df

,Text,labels
0,அர்யவுக்கு நல்ல வாய்ப்பு சிங்கம் சூரியா அண்ணவோ...,0
1,பள்ளியோ கல்லூரியோ படித்துக்கொண்டிருக்கும்போது ...,0
2,தல தல தல தல தல தல தல தல தல தல தல தல தல தல தல த...,0
3,best teamsuper kadhai karuippa irukura ponnuga...,0
5,apadilam nadakathu nadakavum koodathu mass dia...,0
...,...,...
29565,ajth sar mass vasanam ner konda baarvai mass,0
29566,adei anga enpt trailer mukkitu,1
29567,kv anandh sir sentiment ayan maatran kavan maa...,0
29568,ஒவ்வொருத்தருக்கும் ஒவ்வொரு பைத்தியம் நமக்கு யு...,0


In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/sarcasm/sarcasm_tam_test_without_labels.csv')
df1

,ID,Text
0,Id_01,Kangana wow awesome yr ye lakdi sbae alh hai
1,Id_02,விழுப்புரம் வன்னிய கவுண்டர் சார்பாக வாழ்த்துக...
2,Id_03,திரௌபதி திரைப்படம் வெற்றி பெற வாணியர் சமுதாயம்...
3,Id_04,"இந்த திரைப்படம் வெற்றிபெற, ஆதி தமிழன் அதாவது இ..."
4,Id_05,dai thala pera sonnalay summa tamil naday athi...
...,...,...
6333,Id_6334,NTR _ Ajith mutuals like here
6334,Id_6335,aiyo #thala marana mass #thala love you so muc...
6335,Id_6336,Yan kadavula I love you thala
6336,Id_6337,Thank you vijay sethupathi....for acted at syr...


In [ ]:
from polyglot.detect import Detector

def ling(x):
   # Handle empty or very short strings
   if x is None or len(x.strip()) < 10:  # Adjust the threshold as needed
       return 'unknown'
   try:
       detector = Detector(x)
       return detector.language.name
   except:
       return 'unknown'  # Return 'unknown' if language detection fails

df1['lang'] = df1['Text'].apply(lambda x: ling(x))

In [ ]:
l=df1['lang'].unique()
for x in l:
  print(x)

Kinyarwanda
Tamil
English
Oromo
Interlingue
Norwegian Nynorsk
Khasi
Faroese
Shona
Nyanja
Sanskrit
Wolof
unknown
Scottish Gaelic
Indonesian
Estonian
Manx
Samoan
Venda
Zulu
Latin
Danish
Xhosa
Malayalam
Fijian
Rundi
Malagasy
Scots
Afar
Galician
Tongan
Javanese
Irish
Corsican
Kalaallisut
Northern Sotho
Somali
zzp
Klingon
Uzbek
Spanish
Malay
Serbian
Nauru
Vietnamese
Southern Sotho
Azerbaijani
Tagalog
Finnish
Albanian
Slovak
Sundanese
Māori
Quechua
Lithuanian
Ganda
Aymara
Interlingua
Hungarian
Haitian Creole
Inupiaq
Icelandic
Welsh
Tsonga
Romanian
Hausa
Basque
Yoruba
Portuguese
Swedish
Lingala
Zhuang
Guarani
Swahili
Polish
Romansh
Czech
Norwegian
Dutch
Afrikaans
Morisyen
Slovenian
Hawaiian
Igbo
Occitan
Akan
Swati
Arabic
Telugu
Hmong
Turkish
Tswana
Western Frisian
Waray
Latvian
Russian
Italian
Punjabi
Turkmen
Bislama
French
Tatar
Esperanto


In [ ]:

df1[['Text','lang']]


,Text,lang
0,Kangana wow awesome yr ye lakdi sbae alh hai,Kinyarwanda
1,விழுப்புரம் வன்னிய கவுண்டர் சார்பாக வாழ்த்துக...,Tamil
2,திரௌபதி திரைப்படம் வெற்றி பெற வாணியர் சமுதாயம்...,Tamil
3,"இந்த திரைப்படம் வெற்றிபெற, ஆதி தமிழன் அதாவது இ...",Tamil
4,dai thala pera sonnalay summa tamil naday athi...,English
...,...,...
6333,NTR _ Ajith mutuals like here,English
6334,aiyo #thala marana mass #thala love you so muc...,English
6335,Yan kadavula I love you thala,English
6336,Thank you vijay sethupathi....for acted at syr...,English


In [ ]:
df1[df1['lang']=='Esperanto']
df1.loc[5715,'lang']='English'
df1[df1['lang']=='Esperanto']
df1.loc[5716,'Text']

'Teaser na ipidiirukanum Mass teaserFirst day first show pakamavidamaden'

In [ ]:
def preprocess_text(text):
    # Convert English text to lowercase
    text = re.sub(r'[A-Za-z]+', lambda x: x.group(0).lower(), text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove punctuations (excluding Tamil characters)
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', text)

    # Remove emojis
    text = re.sub(r'[^\w\s\u0B80-\u0BFF]', '', text, flags=re.UNICODE)

    return text

# Apply preprocessing to the 'Text' column
df1['Text'] = df1['Text'].apply(preprocess_text)

# Display the first few rows of the dataframe to verify preprocessing
df1

,ID,Text,lang
0,Id_01,kangana wow awesome yr ye lakdi sbae alh hai,Kinyarwanda
1,Id_02,விழுப்புரம் வன்னிய கவுண்டர் சார்பாக வாழ்த்துக...,Tamil
2,Id_03,திரௌபதி திரைப்படம் வெற்றி பெற வாணியர் சமுதாயம்...,Tamil
3,Id_04,இந்த திரைப்படம் வெற்றிபெற ஆதி தமிழன் அதாவது இந...,Tamil
4,Id_05,dai thala pera sonnalay summa tamil naday athi...,English
...,...,...,...
6333,Id_6334,ntr _ ajith mutuals like here,English
6334,Id_6335,aiyo thala marana mass thala love you so much ...,English
6335,Id_6336,yan kadavula i love you thala,English
6336,Id_6337,thank you vijay sethupathifor acted at syraaaa...,English


In [ ]:
df1[df1['lang']=='Tatar']
df1.loc[[5478,5959],'lang']='English'
df1[df1['lang']=='Tatar']
df1[df1['lang']=='Bislama']
df1.loc[5163,'lang']='English'
df1[df1['lang']=='Bislama']
df1.loc[6106,'lang']='Hindi'
df1[df1['lang']=='Turkmen']
df1.loc[4605,'lang']='English'
df1[df1['lang']=='Turkmen']
df1[df1['lang']=='Punjabi']
df1[df1['lang']=='Italian']
df1.loc[[4364,4973],'lang']='English'
df1[df1['lang']=='Italian']
df1[df1['lang']=='Russian']
df1[df1['lang']=='Latvian']
df1.loc[[3856,5329],'lang']='English'
df1[df1['lang']=='Latvian']
df1[df1['lang']=='Waray']
df1.loc[[3621,5631,5653],'lang']='English'
df1[df1['lang']=='Waray']
df1[df1['lang']=='Western Frisian']
df1.loc[[3541,6315],"lang"]='English'
df1[df1['lang']=='Western Frisian']
df1[df1['lang']=='Tswana']
df1.loc[[3470,6238],"lang"]='English'
df1[df1['lang']=='Tswana']
df1[df1['lang']=='Turkish']
df1.loc[[3160,5561],"lang"]="English"
df1.loc[df1['lang']=='Turkish',"Text"]='Well, movies 1 and 2 were great, 3 is shit'
df1.loc[df1['lang']=='Turkish',"lang"]="English"
df1[df1['lang']=='Turkish']
df1[df1['lang']=='Hmong']
df1.loc[3132,'lang']='English'
df1[df1['lang']=='Telugu']
df1.loc[2862,'Text']='Nana talaga wadana kana naka e sanama baga nachachada'
df1[df1['lang']=='Telugu']
df1.loc[4284,'Text']='Rajanakata Talaga Fayanasa Laka Kattada'
df1[df1['lang']=='Telugu']
df1[df1['lang']=='Swati']
df1.loc[2708,'lang']='English'
df1[df1['lang']=='Swati']



,ID,Text,lang


In [ ]:
df1[df1['lang']=='Akan']
df1.loc[[2696, 5324], 'lang'] = 'English'
df1[df1['lang']=='Akan']
df1.loc[3273, 'Text'] = 'Alluarjun fan got 15k likes in Devava 3 miss'
df1[df1['lang']=='Akan']
df1.loc[3273, 'lang'] = 'English'
df1[df1['lang']=='Akan']
df1[df1['lang']=='Occitan']
df1.loc[2496, 'lang'] = 'English'
df1[df1['lang']=='Occitan']
df1[df1['lang']=='Igbo']
df1.loc[[1969, 3104, 3216], 'lang'] = 'English'
df1[df1['lang']=='Igbo']
df1[df1['lang']=='Hawaiian']
df1.loc[1874, 'Text'] = 'Write a heartfelt like for Kangana Ranaut'
df1.loc[1874, 'lang'] = 'English'
df1.loc[df1['lang']=='Hawaiian', 'lang'] = 'English'
df1[df1['lang']=='Hawaiian']
df1[df1['lang']=='Slovenian']
df1.loc[[1872, 3553], 'lang'] = 'English'
df1[df1['lang']=='Slovenian']
df1[df1['lang']=='Morisyen']
df1.loc[[1757, 3891], 'lang'] = 'English'
df1[df1['lang']=='Morisyen']
df1[df1['lang']=='Afrikaans']
df1.loc[1728, 'lang'] = 'English'
df1[df1['lang']=='Afrikaans']
df1[df1['lang']=='Dutch']
df1.loc[1720, 'lang'] = 'English'
df1[df1['lang']=='Dutch']
df1[df1['lang']=='Norwegian']
df1.loc[df1['lang']=='Norwegian', 'lang'] = 'English'
df1[df1['lang']=='Norwegian']
df1[df1['lang']=='Czech']
df1.loc[df1['lang']=='Czech', 'lang'] = 'English'
df1[df1['lang']=='Czech']
df1[df1['lang']=='Romansh']
df1.loc[1563, 'lang'] = 'English'
df1[df1['lang']=='Romansh']
df1[df1['lang']=='Polish']
df1.loc[[1388, 3412], 'lang'] = 'English'
df1[df1['lang']=='Polish']
df1[df1['lang']=='Swahili']
df1.loc[df1['lang']=='Swahili', 'lang'] = 'English'
df1[df1['lang']=='Swahili']
df1[df1['lang']=='Guarani']
df1.loc[df1['lang']=='Guarani', 'lang'] = 'English'
df1[df1['lang']=='Guarani']
df1[df1['lang']=='Zhuang']
df1.loc[1372, 'lang'] = 'English'
df1[df1['lang']=='Zhuang']
df1[df1['lang']=='Lingala']
df1.loc[1358, 'Text'] = 'This song is right for PUBG'
df1.loc[[1358, 1843], 'lang'] = 'English'
df1[df1['lang']=='Lingala']
df1[df1['lang']=='Swedish']
df1.loc[df1['lang']=='Swedish', 'lang'] = 'English'
df1[df1['lang']=='Swedish']
df1[df1['lang']=='Portuguese']
df1.loc[2625, 'Text'] = 'I do not even know how I ended up here'
df1.loc[df1['lang']=='Portuguese', 'lang'] = 'English'
df1[df1['lang']=='Portuguese']
df1[df1['lang']=='Yoruba']
df1.loc[df1['lang']=='Yoruba', 'lang'] = 'English'
df1[df1['lang']=='Yoruba']


,ID,Text,lang


In [ ]:


df1[df1['lang'] == 'Basque']
df1.loc[2104, 'Text'] = 'There is not as much plastic in childrens dolls as there is in their makeup'
df1[df1['lang'] == 'Basque']
df1.loc[[1156, 2104], 'lang'] = 'English'
df1[df1['lang'] == 'Basque']
df1[df1['lang'] == 'Hausa']
df1.loc[df1['lang'] == 'Hausa', 'lang'] = 'English'
df1[df1['lang'] == 'Hausa']
df1[df1['lang'] == 'Romanian']
df1.loc[[1140, 1870, 1950, 4561], 'lang'] = 'English'
df1[df1['lang'] == 'Romanian']
df1[df1['lang'] == 'Tsonga']
df1.loc[df1['lang'] == 'Tsonga', 'lang'] = 'English'
df1[df1['lang'] == 'Tsonga']
df1[df1['lang'] == 'Welsh']
df1.loc[[979, 1545, 1551, 1981, 3061], 'lang'] = 'English'
df1[df1['lang'] == 'Welsh']
df1[df1['lang'] == 'Icelandic']
df1.loc[df1['lang'] == 'Icelandic', 'lang'] = 'English'
df1[df1['lang'] == 'Icelandic']
df1[df1['lang'] == 'Inupiaq']
df1.loc[891, 'lang'] = 'English'
df1[df1['lang'] == 'Inupiaq']
df1[df1['lang'] == 'Haitian Creole']
df1.loc[[864, 4562], 'lang'] = 'English'
df1[df1['lang'] == 'Haitian Creole']
df1[df1['lang'] == 'Hungarian']
df1.loc[df1['lang'] == 'Hungarian', 'lang'] = 'English'
df1[df1['lang'] == 'Hungarian']
df1[df1['lang'] == 'Interlingua']
df1.loc[842, 'lang'] = 'English'
df1[df1['lang'] == 'Interlingua']
df1[df1['lang'] == 'Ganda']
df1.loc[1627, 'Text'] = 'Hey loser, what can you see?'
df1.loc[2689, 'Text'] = 'what a bitch she is'
df1.loc[df1['lang'] == 'Ganda', 'lang'] = 'English'
df1[df1['lang'] == 'Ganda']
df1[df1['lang'] == 'Lithuanian']
df1.loc[df1['lang'] == 'Lithuanian', 'lang'] = 'English'
df1[df1['lang'] == 'Lithuanian']
df1[df1['lang'] == 'Quechua']
df1.loc[df1['lang'] == 'Quechua', 'lang'] = 'English'
df1[df1['lang'] == 'Quechua']


,ID,Text,lang


In [ ]:


df1[df1['lang'] == 'Māori']
df1.loc[df1['lang'] == 'Māori', 'lang'] = 'English'
df1[df1['lang'] == 'Māori']
df1[df1['lang'] == 'Sundanese']
df1.loc[556, 'Text'] = 'The most stupid stupid action movie in the world'
df1.loc[3646, 'Text'] = 'Those who abused Kangana Ranaut after seeing the trailer of Librandoo have probably got a fire in their ass. You fool yourself and do not know why you are abusing, you will just copy your logo if someone has abused or abused you.'
df1[df1['lang'] == 'Sundanese']
df1.loc[4045, 'Text'] = 'Sorry but when Kangana Ranauts entry came, I felt half nude and in prosthetic makeup to look like an animated character, my sister trolled Taapsee and Bhumi, now what to say about Kangnas makeup'
df1[df1['lang'] == 'Sundanese']
df1.loc[df1['lang'] == 'Sundanese', 'lang'] = 'English'
df1[df1['lang'] == 'Sundanese']
df1[df1['lang'] == 'Slovak']
df1.loc[3874, 'Text'] = 'I did not understand what you were saying'
df1.loc[df1['lang'] == 'Slovak', 'lang'] = 'English'
df1[df1['lang'] == 'Slovak']
df1[df1['lang'] == 'Albanian']
df1.loc[df1['lang'] == 'Albanian', 'lang'] = 'English'
df1[df1['lang'] == 'Albanian']
df1[df1['lang'] == 'Finnish']
df1.loc[df1['lang'] == 'Finnish', 'lang'] = 'English'
df1[df1['lang'] == 'Finnish']
df1[df1['lang'] == 'Tagalog']
df1.loc[df1['lang'] == 'Tagalog', 'lang'] = 'English'
df1[df1['lang'] == 'Tagalog']
df1[df1['lang'] == 'Azerbaijani']
df1.loc[df1['lang'] == 'Azerbaijani', 'lang'] = 'English'
df1[df1['lang'] == 'Azerbaijani']
df1[df1['lang'] == 'Southern Sotho']
df1.loc[[444, 3051, 3827], 'lang'] = 'English'
df1[df1['lang'] == 'Southern Sotho']
df1[df1['lang'] == 'Vietnamese']
df1.loc[420, 'Text'] = 'Hello everyone'
df1.loc[1070, 'Text'] = 'It is not even day in the trailer'
df1.loc[[420, 1070], 'lang'] = 'English'
df1[df1['lang'] == 'Vietnamese']
df1[df1['lang'] == 'Nauru']
df1.loc[[396, 623], 'lang'] = 'English'
df1[df1['lang'] == 'Nauru']
df1[df1['lang'] == 'Serbian']
df1.loc[373, 'lang'] = 'Serbian'
df1.loc[373, 'Text'] = 'Who from Russians like the comment'
df1[df1['lang'] == 'Serbian']
df1.loc[[373, 3277], 'lang'] = 'English'
df1[df1['lang'] == 'Serbian']
df1[df1['lang'] == 'Malay']
df1.loc[df1['lang'] == 'Malay', 'lang'] = 'English'
df1[df1['lang'] == 'Malay']
df1.loc[df1['lang'] == 'Spanish', 'Text'] = 'when are they going to start making subtitles'
df1.loc[329, 'lang'] = 'English'
df1[df1['lang'] == 'Spanish']
df1[df1['lang'] == 'Uzbek']
df1.loc[df1['lang'] == 'Uzbek', 'lang'] = 'English'
df1[df1['lang'] == 'Uzbek']
df1[df1['lang'] == 'Klingon']
df1.loc[df1['lang'] == 'Klingon', 'lang'] = 'English'
df1[df1['lang'] == 'Klingon']
df1[df1['lang'] == 'zzp']
df1.loc[df1['lang'] == 'zzp', 'lang'] = 'English'
df1[df1['lang'] == 'zzp']
df1.loc[df1['lang'] == 'Somali', 'lang'] = 'English'
df1[df1['lang'] == 'Somali']
df1[df1['lang'] == 'Northern Sotho']
df1.loc[df1['lang'] == 'Northern Sotho', 'lang'] = 'English'
df1[df1['lang'] == 'Northern Sotho']
df1[df1['lang'] == 'Kalaallisut']
df1.loc[2001, 'Text'] = 'I saw it in the beginning, did you guys also see it'
df1[df1['lang'] == 'Kalaallisut']
df1.loc[df1['lang'] == 'Kalaallisut', 'lang'] = 'English'
df1[df1['lang'] == 'Kalaallisut']


,ID,Text,lang


In [ ]:
df1[df1['lang'] == 'Corsican']
df1.loc[df1['lang'] == 'Corsican', 'lang'] = 'English'
df1[df1['lang'] == 'Corsican']
df1[df1['lang'] == 'Irish']
df1.loc[df1['lang'] == 'Irish', 'lang'] = 'English'
df1[df1['lang'] == 'Irish']
df1[df1['lang'] == 'Javanese']
df1.loc[5221, 'Text'] = 'Rajinikanth telugu fans are a like escond'
df1.loc[df1['lang'] == 'Javanese', 'lang'] = 'English'
df1[df1['lang'] == 'Javanese']
df1[df1['lang'] == 'Tongan']
df1.loc[6193, 'Text'] = 'The trailer pales in comparison to Zero'
df1.loc[df1['lang'] == 'Tongan', 'lang'] = 'English'
df1[df1['lang'] == 'Tongan']
df1[df1['lang'] == 'Galician']
df1.loc[3831, 'lang'] = 'Tamil'
df1.loc[4859, 'Text'] = 'Dajjal has come into the world and is Khan'
df1.loc[df1['lang'] == 'Galician', 'lang'] = 'English'
df1[df1['lang'] == 'Galician']
df1[df1['lang'] == 'Afar']
df1.loc[df1['lang'] == 'Afar', 'lang'] = 'English'
df1[df1['lang'] == 'Afar']
df1[df1['lang'] == 'Scots']
df1.loc[136, 'Text'] = 'Get down Matt you are beggars asking for all the likes'
df1.loc[1450, 'Text'] = 'Do not go down, she is asking for it'
df1.loc[df1['lang'] == 'Scots', 'lang'] = 'English'
df1[df1['lang'] == 'Scots']
df1[df1['lang'] == 'Malagasy']
df1.loc[5131, 'Text'] = 'There is a joke going on here'
df1.loc[df1['lang'] == 'Malagasy', 'lang'] = 'English'
df1[df1['lang'] == 'Malagasy']
df1[df1['lang'] == 'Rundi']
df1.loc[df1['lang'] == 'Rundi', 'lang'] = 'English'
df1[df1['lang'] == 'Rundi']
df1[df1['lang'] == 'Fijian']
df1.loc[df1['lang'] == 'Fijian', 'lang'] = 'English'
df1[df1['lang'] == 'Fijian']
df1[df1['lang'] == 'Malayalam']
df1.loc[103, 'Text'] = 'malayali suriya fan malayali suriya fan'
df1[df1['lang'] == 'Malayalam']
df1.loc[1075, 'Text'] = 'waiting for thalapathy myskin combo a dictative movie like sherlock homes ananganara airera vannal engananarakaka'
df1[df1['lang'] == 'Malayalam']
df1.loc[3830, 'Text'] = 'Laladdan mohanlal singam surya'
df1[df1['lang'] == 'Malayalam']
df1.loc[4904, 'Text'] = 'suriya and lal rock rakaka'
df1[df1['lang'] == 'Malayalam']
df1.loc[df1['lang'] == 'Malayalam', 'lang'] = 'English'
df1[df1['lang'] == 'Malayalam']
df1[df1['lang'] == 'Xhosa']
df1.loc[df1['lang'] == 'Xhosa', 'lang'] = 'English'
df1[df1['lang'] == 'Xhosa']
df1[df1['lang'] == 'Danish'].head(60)
df1.loc[4027, "Text"] = 'What, why, how and to whom do you want to show: These are some of the questions in my heart'
df1[df1['lang'] == 'Danish'].tail(20)
df1.loc[df1['lang'] == 'Danish', 'lang'] = 'English'
df1[df1['lang'] == 'Danish']
df1[df1['lang'] == 'Latin']
df1.loc[df1['lang'] == 'Latin', 'lang'] = 'English'
df1[df1['lang'] == 'Latin']
df1[df1['lang'] == 'Zulu']
df1.loc[df1['lang'] == 'Zulu', 'lang'] = 'English'
df1[df1['lang'] == 'Zulu']
df1[df1['lang'] == 'Venda']
df1.loc[df1['lang'] == 'Venda', 'lang'] = 'English'
df1[df1['lang'] == 'Venda']
df1[df1['lang'] == 'Samoan']
df1.loc[df1['lang'] == 'Samoan', 'lang'] = 'English'
df1[df1['lang'] == 'Samoan']
df1[df1['lang'] == 'Manx']
df1.loc[df1['lang'] == 'Manx', 'lang'] = 'English'
df1[df1['lang'] == 'Manx']


,ID,Text,lang


In [ ]:
df1[df1['lang'] == 'Estonian']
df1.loc[5069, 'Text'] = 'Those who have seen ink like Telugu Vijay fans'
df1[df1['lang'] == 'Estonian']
df1.loc[df1['lang'] == 'Estonian', 'lang'] = 'English'
df1[df1['lang'] == 'Estonian']
df1[df1['lang'] == 'Indonesian']
df1.loc[6288, 'Text'] = 'you have to wait a little longer to know that I am semma vikram sir malay le pinringaaa'
df1[df1['lang'] == 'Indonesian']
df1.loc[6303, 'Text'] = 'u have to wait a little longer to know me'
df1[df1['lang'] == 'Indonesian']
df1.loc[df1['lang'] == 'Indonesian', 'lang'] = 'English'
df1[df1['lang'] == 'Indonesian']
df1[df1['lang'] == 'Scottish Gaelic']
df1.loc[df1['lang'] == 'Scottish Gaelic', 'lang'] = 'English'
df1[df1['lang'] == 'Scottish Gaelic']
df1[df1['lang'] == 'unknown']
df1.loc[1672, 'Text'] = 'pwli shop anyayam'
df1.loc[2759, 'Text'] = '786 k likes mashaallah thala thala masasa'
df1.loc[2918, 'lang'] = 'Tamil'
df1.loc[3824, 'lang'] = 'Tamil'
df1.loc[[4320, 5546], 'lang'] = 'Tamil'
df1.loc[4745, 'Text'] = 'What is this doing in my trends'
df1.loc[6170, 'Text'] = 'rajini jai thalaivaa, the father of style'
df1[df1['lang'] == 'unknown']
df1.loc[df1['lang'] == 'unknown', 'lang'] = 'English'
df1[df1['lang'] == 'unknown']
df1[df1['lang'] == 'Wolof']
df1.loc[2051, 'Text'] = 'i did not understand anything but i enjoyed it'
df1.loc[df1['lang'] == 'Wolof', 'lang'] = 'English'
df1[df1['lang'] == 'Wolof']
df1[df1['lang'] == 'Sanskrit']
df1.loc[df1['lang'] == 'Sanskrit', 'lang'] = 'English'
df1[df1['lang'] == 'Sanskrit']
df1[df1['lang'] == 'Nyanja']
df1.loc[df1['lang'] == 'Nyanja', 'lang'] = 'English'
df1[df1['lang'] == 'Nyanja']
df1[df1['lang'] == 'Shona']
df1.loc[df1['lang'] == 'Shona', 'lang'] = 'English'
df1[df1['lang'] == 'Shona']
df1[df1['lang'] == 'Faroese']
df1.loc[df1['lang'] == 'Faroese', 'lang'] = 'English'
df1[df1['lang'] == 'Faroese']
df1[df1['lang'] == 'Khasi']
df1.loc[df1['lang'] == 'Khasi', 'lang'] = 'English'
df1[df1['lang'] == 'Khasi']


,ID,Text,lang


In [ ]:
df1[df1['lang'] == 'Norwegian Nynorsk']
df1.loc[df1['lang'] == 'Norwegian Nynorsk', 'lang'] = 'English'
df1.loc[df1['lang'] == 'Interlingue', 'lang'] = 'English'
df1.loc[df1['lang'] == 'Oromo', 'lang'] = 'English'
df1[df1['lang'] == 'Oromo']
df1[df1['lang'] == 'Interlingue']
df1[df1['lang'] == 'Kinyarwanda']
df1.loc[2072, 'Text'] = 'Neither looks like Kangana Ranaut nor Amma'
df1.loc[df1['lang'] == 'Kinyarwanda', 'lang'] = 'English'
df1[df1['lang'] == 'Kinyarwanda']


,ID,Text,lang


In [ ]:
l=df1['lang'].unique()
for x in l:
  print(x)

English
Tamil
Aymara
Arabic
Telugu
Russian
Punjabi
French
Hindi


In [ ]:
df1[df1['lang'] == 'Arabic']
df1.loc[2785, 'Text'] = 'Hey I just made an amv video Give me your opinion about the video Please support me My channel is new and whoever will subscribe to my channel, I will give him the name of my Facebook account and I will give him my picture as he wants.'
df1.loc[4901, 'Text'] = 'God gave me twins Rawan and Razan. Come in and see them'
df1[df1['lang'] == 'Arabic']
df1.loc[4969, 'Text'] = 'Are you an idea maker or a filmmaker Enter this link for only two days to subscribe'
df1[df1['lang'] == 'Arabic']
df1.loc[6245, 'Text'] = 'Taxi Rides series, Episode 6, Episode title: His response story, but Amman Media Foundation for Artistic Production, written and directed by Ibrahim Al-Khamisi. To receive the next episode of the series, please subscribe to the channel and activate the bell button. Channel link.'
df1[df1['lang'] == 'Arabic']
df1.loc[df1['lang'] == 'Arabic', 'lang'] = 'English'
df1[df1['lang'] == 'Arabic']
df1[df1['lang'] == 'Aymara']
df1.loc[df1['lang'] == 'Aymara', 'lang'] = 'English'
df1[df1['lang'] == 'Aymara']
df1.loc[df1['lang'] == 'Punjabi', 'Text'] = 'Patt par lagad in the subj sun'
df1.loc[4560, 'lang'] = 'English'
df1[df1['lang'] == 'Punjabi']
df1[df1['lang'] == 'Russian']
df1.loc[4262, 'Text'] = 'I watched this movie, it was very cool'
df1.loc[5790, 'Text'] = 'READ THE NEWS AFFILIATE PROGRAM FINANCE FAQ ABOUT THE COMPANY READ THE NEWS YOU ARE READING THE NEWS'
df1.loc[[4262, 5790], 'lang'] = 'English'
df1[df1['lang'] == 'Russian']
df1.loc[df1['lang'] == 'Telugu', 'lang'] = 'English'
df1[df1['lang'] == 'Telugu']
df1[df1['lang'] == 'French']
df1.loc[5278, 'Text'] = 'ahh I was waiting for this teaser so much'
df1.loc[5278, 'lang'] = 'English'
df1[df1['lang'] == 'French']
df1[df1['lang'] == 'Hindi']
df1.loc[6106, 'Text'] = 'Who came from Tiktok to listen to this BS tone'
df1.loc[6106, 'lang'] = 'English'



In [ ]:
l=df1['lang'].unique()
for x in l:
  print(x)

English
Tamil


In [ ]:
df1

,ID,Text,lang
0,Id_01,kangana wow awesome yr ye lakdi sbae alh hai,English
1,Id_02,விழுப்புரம் வன்னிய கவுண்டர் சார்பாக வாழ்த்துக...,Tamil
2,Id_03,திரௌபதி திரைப்படம் வெற்றி பெற வாணியர் சமுதாயம்...,Tamil
3,Id_04,இந்த திரைப்படம் வெற்றிபெற ஆதி தமிழன் அதாவது இந...,Tamil
4,Id_05,dai thala pera sonnalay summa tamil naday athi...,English
...,...,...,...
6333,Id_6334,ntr _ ajith mutuals like here,English
6334,Id_6335,aiyo thala marana mass thala love you so much ...,English
6335,Id_6336,yan kadavula i love you thala,English
6336,Id_6337,thank you vijay sethupathifor acted at syraaaa...,English


In [ ]:
def preprocess_text(text):
    # Convert English text to lowercase
    text = re.sub(r'[A-Za-z]+', lambda x: x.group(0).lower(), text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove punctuations (excluding Tamil characters)
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', text)

    # Remove emojis
    text = re.sub(r'[^\w\s\u0B80-\u0BFF]', '', text, flags=re.UNICODE)

    return text

# Apply preprocessing to the 'Text' column
df1['Text'] = df1['Text'].apply(preprocess_text)

# Display the first few rows of the dataframe to verify preprocessing

In [ ]:
df1

,ID,Text,lang
0,Id_01,kangana wow awesome yr ye lakdi sbae alh hai,English
1,Id_02,விழுப்புரம் வன்னிய கவுண்டர் சார்பாக வாழ்த்துக...,Tamil
2,Id_03,திரௌபதி திரைப்படம் வெற்றி பெற வாணியர் சமுதாயம்...,Tamil
3,Id_04,இந்த திரைப்படம் வெற்றிபெற ஆதி தமிழன் அதாவது இந...,Tamil
4,Id_05,dai thala pera sonnalay summa tamil naday athi...,English
...,...,...,...
6333,Id_6334,ntr _ ajith mutuals like here,English
6334,Id_6335,aiyo thala marana mass thala love you so much ...,English
6335,Id_6336,yan kadavula i love you thala,English
6336,Id_6337,thank you vijay sethupathifor acted at syraaaa...,English


In [ ]:
stop_words = [
    "எந்த", "என்று", "எனவே", "ஏனெனில்", "அங்கு", "அதில்", "அவர்", "அவ்வாறு", "அல்லது", "அவர்கள்",
    "அவை", "ஆக", "இங்கு", "இதை", "இதைத்", "இது", "இதில்", "இந்த", "இந்தப்", "இதற்கு", "இதனால்",
    "இவைகள்", "இவர்கள்", "இவை", "இவை", "உள்ள", "என", "ஏன்", "எப்பொழுது", "எப்போது", "எது", "எது",
    "ஒரு", "ஒன்றின்", "ஒரே", "உள்ள", "மற்றும்", "மிக", "மிகவும்", "வெளியே", "என்பதை", "என்பது",
    "என்கிற", "என்று", "சில", "சிலரின்", "மற்றும்", "சிறிய", "சிறிது", "சிறு", "தன்", "தனது",
    "இருக்கும்", "பின்", "பின்னர்", "மீண்டும்", "ஆனால்", "அதனால்", "போல்", "மேலும்", "மேலேயே",
    "அதற்கு", "அதை", "என்ன", "என்றால்", "அவரது", "உங்கள்", "நான்", "எனக்கு", "எனது", "நீங்கள்"
]

In [ ]:
import spacy
from spacy.lang.ta import Tamil

# Create a custom Tamil stop words list
tamil_stop_words = set(stop_words)

# Load a Tamil language model
nlp = Tamil()

# Extend the spaCy stop word list with custom Tamil stop words
for word in tamil_stop_words:
    nlp.vocab[word].is_stop = True

print(tamil_stop_words)


{'மீண்டும்', 'ஆனால்', 'சிறிய', 'அதில்', 'இங்கு', 'என்பது', 'எனவே', 'போல்', 'அவ்வாறு', 'உள்ள', 'எனது', 'என்று', 'மிக', 'ஏன்', 'மேலேயே', 'ஏனெனில்', 'இவை', 'வெளியே', 'இவைகள்', 'மேலும்', 'என்றால்', 'அவர்', 'இதைத்', 'இது', 'ஒரு', 'மற்றும்', 'பின்', 'ஒன்றின்', 'அதை', 'அவரது', 'என', 'இந்தப்', 'எப்போது', 'சிறிது', 'தன்', 'பின்னர்', 'இந்த', 'அதற்கு', 'நீங்கள்', 'இருக்கும்', 'இதை', 'இதில்', 'எந்த', 'எப்பொழுது', 'அதனால்', 'சிலரின்', 'இதனால்', 'என்கிற', 'ஆக', 'எனக்கு', 'என்பதை', 'ஒரே', 'சிறு', 'என்ன', 'அங்கு', 'அல்லது', 'அவர்கள்', 'சில', 'தனது', 'இதற்கு', 'உங்கள்', 'மிகவும்', 'இவர்கள்', 'எது', 'அவை', 'நான்'}


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Get the list of Tamil stopwords
stop_words = tamil_stop_words

def remove_stopwords(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Reconstruct the text
    return ' '.join(filtered_tokens)

# Apply the function to your DataFrame
df1['Text'] = df1['Text'].apply(remove_stopwords)

# Display the cleaned DataFrame
df1


,ID,Text,lang
0,Id_01,kangana wow awesome yr ye lakdi sbae alh hai,English
1,Id_02,விழுப்புரம் வன்னிய கவுண்டர் சார்பாக வாழ்த்துக்கள்,Tamil
2,Id_03,திரௌபதி திரைப்படம் வெற்றி பெற வாணியர் சமுதாயம்...,Tamil
3,Id_04,திரைப்படம் வெற்றிபெற ஆதி தமிழன் அதாவது சமுதாயம...,Tamil
4,Id_05,dai thala pera sonnalay summa tamil naday athi...,English
...,...,...,...
6333,Id_6334,ntr _ ajith mutuals like here,English
6334,Id_6335,aiyo thala marana mass thala love you so much ...,English
6335,Id_6336,yan kadavula i love you thala,English
6336,Id_6337,thank you vijay sethupathifor acted at syraaaa...,English


In [ ]:
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list.extend(['ena','unaku','per','irukanga','panna','yarum','mattum','ivan','ada','pesa','unakku','k','sari','idhu','vida','vittu','enga','yen','ithu','poda','dey','irundhu','ya','la', 'u','r','s','bro','da','dei','dai','nu','ah','nee','ni','illa','un','ok','na','pls','ur','unga'])
stopwords_list.extend(['🥰','indha','antha','vera','iruka','pola','innum','avan','summa','ellam','thaan','romba','❤️','ana','ama','apdi','ithula','po','evlo','eruku','irukum','nama','enna','va','hi','h','ku','iruku','naa','va','oru','athu','avanga','neenga','tha','en','di','dhan','ne','ella','intha'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stopwords_list]
    # Reconstruct the text
    return ' '.join(filtered_tokens)

# Apply the function to your DataFrame
df1['Text'] = df1['Text'].apply(remove_stopwords)

# Display the cleaned DataFrame
df1

,ID,Text,lang
0,Id_01,kangana wow awesome yr ye lakdi sbae alh hai,English
1,Id_02,விழுப்புரம் வன்னிய கவுண்டர் சார்பாக வாழ்த்துக்கள்,Tamil
2,Id_03,திரௌபதி திரைப்படம் வெற்றி பெற வாணியர் சமுதாயம்...,Tamil
3,Id_04,திரைப்படம் வெற்றிபெற ஆதி தமிழன் அதாவது சமுதாயம...,Tamil
4,Id_05,thala pera sonnalay tamil naday athirum daa,English
...,...,...,...
6333,Id_6334,ntr _ ajith mutuals like,English
6334,Id_6335,aiyo thala marana mass thala love much thala,English
6335,Id_6336,yan kadavula love thala,English
6336,Id_6337,thank vijay sethupathifor acted syraaaaaaaaaa,English


In [ ]:
df=df.drop(columns=['lang'])
df

,Text,labels
0,அர்யவுக்கு நல்ல வாய்ப்பு சிங்கம் சூரியா அண்ணவோ...,Non-sarcastic
1,பள்ளியோ கல்லூரியோ படித்துக்கொண்டிருக்கும்போது ...,Non-sarcastic
2,தல தல தல தல தல தல தல தல தல தல தல தல தல தல தல த...,Non-sarcastic
3,best teamsuper kadhai karuippa irukura ponnuga...,Non-sarcastic
5,apadilam nadakathu nadakavum koodathu mass dia...,Non-sarcastic
...,...,...
29565,ajth sar mass vasanam ner konda baarvai mass,Non-sarcastic
29566,adei anga enpt trailer mukkitu,Sarcastic
29567,kv anandh sir sentiment ayan maatran kavan maa...,Non-sarcastic
29568,ஒவ்வொருத்தருக்கும் ஒவ்வொரு பைத்தியம் நமக்கு யு...,Non-sarcastic


In [ ]:
df1=df1.drop(columns=['lang'])
df1

,ID,Text
0,Id_01,kangana wow awesome yr ye lakdi sbae alh hai
1,Id_02,விழுப்புரம் வன்னிய கவுண்டர் சார்பாக வாழ்த்துக்கள்
2,Id_03,திரௌபதி திரைப்படம் வெற்றி பெற வாணியர் சமுதாயம்...
3,Id_04,திரைப்படம் வெற்றிபெற ஆதி தமிழன் அதாவது சமுதாயம...
4,Id_05,thala pera sonnalay tamil naday athirum daa
...,...,...
6333,Id_6334,ntr _ ajith mutuals like
6334,Id_6335,aiyo thala marana mass thala love much thala
6335,Id_6336,yan kadavula love thala
6336,Id_6337,thank vijay sethupathifor acted syraaaaaaaaaa


In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
X = df['Text']
y = df['labels']

# Step 2: Create N-grams using CountVectorizer (unigrams, bigrams)
# Adjust range as needed
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X_counts = vectorizer.fit_transform(X)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_counts, y, test_size=0.3, random_state=42)
knn = KNeighborsClassifier(n_neighbors=3)

# Fit the classifier on the training data
knn.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = knn.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print(classification_report(y_test, y_pred))


KeyboardInterrupt: 

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
X = df['Text']
y = df['labels']
vectorizer = CountVectorizer(ngram_range=(1, 3))  # Adjust range as needed
X_counts = vectorizer.fit_transform(X)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_counts, y, test_size=0.3, random_state=42)
mnb = MultinomialNB()

# Fit the classifier on the training data
mnb.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = mnb.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print(classification_report(y_test, y_pred))

# Step 2: Create N-grams using CountVectorizer (unigrams, bigrams)
# Adjust range as needed



Accuracy: 73.97%
              precision    recall  f1-score   support

           0       0.89      0.74      0.81      6485
           1       0.51      0.74      0.60      2342

    accuracy                           0.74      8827
   macro avg       0.70      0.74      0.70      8827
weighted avg       0.79      0.74      0.75      8827



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

# Assuming df is already defined and contains the 'Text' and 'labels' columns
# Example:
# df = pd.DataFrame({'Text': [...], 'labels': [...]})

X = df['Text']
y = df['labels']

# Step 2: Create N-grams using CountVectorizer (unigrams, bigrams, trigrams)
vectorizer = CountVectorizer(ngram_range=(1, 3))
X_counts = vectorizer.fit_transform(X)  # Transform the entire dataset

# Dimensionality Reduction using Truncated SVD
svd = TruncatedSVD(n_components=100)  # Reduce to 100 dimensions
X_reduced = svd.fit_transform(X_counts)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=42)

# Initialize the MLP classifier with adjustments
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)

# Fit the classifier on the training data
mlp.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = mlp.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print(classification_report(y_test, y_pred))


Accuracy: 74.57%
              precision    recall  f1-score   support

           0       0.79      0.89      0.84      6485
           1       0.53      0.34      0.42      2342

    accuracy                           0.75      8827
   macro avg       0.66      0.62      0.63      8827
weighted avg       0.72      0.75      0.73      8827



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

# Assuming df is already defined and contains the 'Text' and 'labels' columns
# Example:
# df = pd.DataFrame({'Text': [...], 'labels': [...]})

X = df['Text']
y = df['labels']

# Step 2: Create N-grams using CountVectorizer (unigrams, bigrams)
vectorizer = CountVectorizer(ngram_range=(1, 3))
X_counts = vectorizer.fit_transform(X)  # Transform the entire dataset

# Dimensionality Reduction using Truncated SVD
svd = TruncatedSVD(n_components=100)  # Reduce to 100 dimensions
X_reduced = svd.fit_transform(X_counts)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=42)

# Initialize the Gradient Boosting classifier
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the classifier on the training data
gbm.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = gbm.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print(classification_report(y_test, y_pred))


Accuracy: 75.79%
              precision    recall  f1-score   support

           0       0.77      0.96      0.85      6485
           1       0.64      0.20      0.30      2342

    accuracy                           0.76      8827
   macro avg       0.71      0.58      0.58      8827
weighted avg       0.74      0.76      0.71      8827



In [ ]:
!pip install xgboost


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

# Assuming df is already defined and contains the 'Text' and 'labels' columns
# Example:
# df = pd.DataFrame({'Text': [...], 'labels': [...]})

X = df['Text']
y = df['labels']

# Step 2: Create N-grams using CountVectorizer (unigrams, bigrams)
vectorizer = CountVectorizer(ngram_range=(1, 2))
X_counts = vectorizer.fit_transform(X)  # Transform the entire dataset

# Dimensionality Reduction using Truncated SVD
svd = TruncatedSVD(n_components=100)  # Reduce to 100 dimensions
X_reduced = svd.fit_transform(X_counts)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=42)

# Initialize the XGBoost classifier
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42, use_label_encoder=False)

# Fit the classifier on the training data
xgb.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = xgb.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print(classification_report(y_test, y_pred))




Accuracy: 75.93%
              precision    recall  f1-score   support

           0       0.77      0.96      0.85      6485
           1       0.65      0.20      0.30      2342

    accuracy                           0.76      8827
   macro avg       0.71      0.58      0.58      8827
weighted avg       0.74      0.76      0.71      8827



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
X = df['Text']
y = df['labels']

# Step 2: Create N-grams using CountVectorizer (unigrams, bigrams)
vectorizer = CountVectorizer(ngram_range=(1, 3))  # Adjust range as needed
X_counts = vectorizer.fit_transform(X)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_counts, y, test_size=0.3, random_state=42)

# Step 4: Fit the Model
model = SVC(kernel='linear')  # Linear kernel is often effective for text classification
model.fit(X_train, y_train)

# Step 5: Make Predictions
y_pred = model.predict(X_test)

# Step 6: Evaluate the Model
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Accuracy: 0.7847513311430837
               precision    recall  f1-score   support

Non-sarcastic       0.81      0.92      0.86      6485
    Sarcastic       0.65      0.42      0.51      2342

     accuracy                           0.78      8827
    macro avg       0.73      0.67      0.69      8827
 weighted avg       0.77      0.78      0.77      8827



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
X = df['Text']
y = df['labels']

# Step 2: Create N-grams using CountVectorizer (unigrams, bigrams)
vectorizer = CountVectorizer(ngram_range=(1, 3))  # Adjust range as needed
X_counts = vectorizer.fit_transform(X)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_counts, y, test_size=0.3, random_state=42)
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

Accuracy: 0.75
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.87      0.84      6485
           1       0.54      0.41      0.46      2342

    accuracy                           0.75      8827
   macro avg       0.67      0.64      0.65      8827
weighted avg       0.73      0.75      0.74      8827



In [ ]:
'''import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
X = df['Text']
y = df['labels']

# Step 2: Create N-grams using CountVectorizer (unigrams, bigrams)
vectorizer = CountVectorizer(ngram_range=(1, 3))  # Adjust range as needed
X_counts = vectorizer.fit_transform(X)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_counts, y, test_size=0.3, random_state=42)
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)'''

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report




vectorizer = CountVectorizer(ngram_range=(1, 3))

# Fit and transform the dataset
X = vectorizer.fit_transform(df['Text'])
y = df['labels']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.7916619463011215
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.94      0.87      6485
           1       0.70      0.38      0.49      2342

    accuracy                           0.79      8827
   macro avg       0.75      0.66      0.68      8827
weighted avg       0.78      0.79      0.77      8827



In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 3))

# Fit and transform the dataset
X = vectorizer.fit_transform(df['Text'])
y = df['labels']


# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

log_clf = LogisticRegression(random_state=42, max_iter=1000)  # Increase max_iter to 1000
dt_clf = DecisionTreeClassifier(random_state=42)
svc_clf = SVC(probability=True, random_state=42)

# Create an ensemble model using VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('dt', dt_clf), ('svc', svc_clf)],
    voting='hard'  # Use 'soft' voting to predict the class label based on the predicted probabilities
)



# Train the ensemble model on the training set
voting_clf.fit(X_train, y_train)

# Make predictions
y_pred = voting_clf.predict(X_test)

# Evaluate the model on the test set
print(f'Test set accuracy: {accuracy_score(y_test, y_pred):.2f}')

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Test set accuracy: 0.79
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.96      0.87      6485
           1       0.73      0.33      0.45      2342

    accuracy                           0.79      8827
   macro avg       0.76      0.64      0.66      8827
weighted avg       0.78      0.79      0.76      8827



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
X = df['Text']
y = df['labels']

# Step 2: Create N-grams using TfidfVectorizer (unigrams, bigrams)
vectorizer = TfidfVectorizer(ngram_range=(1, 3))  # Adjust range as needed
X_tfidf = vectorizer.fit_transform(X)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# Step 4: Fit the Model
model = SVC(kernel='linear')  # Linear kernel is often effective for text classification
model.fit(X_train, y_train)

# Step 5: Make Predictions
y_pred = model.predict(X_test)

# Step 6: Evaluate the Model
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
X = df['Text']
y = df['labels']
vectorizer = CountVectorizer(ngram_range=(1, 3))  # Adjust range as needed
x_train = vectorizer.fit_transform(X)
y_train=y
x1=df1['Text']
x_test = vectorizer.transform(x1)


# Step 3: Split Data
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42)
mnb = MultinomialNB()

# Fit the classifier on the training data
mnb.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = mnb.predict(x_test)
df2=df1
df2['Labels']=y_pred
df2['Labels']=df2['Labels'].apply(lambda x:'Sarcastic' if x==1 else 'Non-sarcastic')
count=0
for a in y_pred:
  if a==1:
    count+=1
print(count)



0


Series([], Name: Labels, dtype: object)

In [ ]:
df2.to_csv("/content/drive/MyDrive/sarcasm/vel_Tamil_run1.csv", index=False)
df2

,Id,Labels
0,Id_01,Non-sarcastic
1,Id_02,Non-sarcastic
2,Id_03,Non-sarcastic
3,Id_04,Non-sarcastic
4,Id_05,Non-sarcastic
...,...,...
6333,Id_6334,Non-sarcastic
6334,Id_6335,Non-sarcastic
6335,Id_6336,Non-sarcastic
6336,Id_6337,Non-sarcastic


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
X = df['Text']
y = df['labels']
vectorizer = CountVectorizer(ngram_range=(1, 3))  # Adjust range as needed
x_train = vectorizer.fit_transform(X)
Y_train=y
x1=df1['Text']
x_test = vectorizer.transform(x1)

# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(x_train, Y_train, test_size=0.3, random_state=42)

# Initialize and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(x_test)
df4=df1
df4['Labels']=y_pred
df4['Labels']=df4['Labels'].apply(lambda x:'Sarcastic' if x==1 else 'Non-sarcastic')
num_sarcastic = (df4['Labels'] == 'Sarcastic').sum()
print(num_sarcastic)
n = (df4['Labels'] == 'Non-sarcastic').sum()
print(n)
df4=df4.drop(columns=['Text'])
df4=df4.rename(columns={'ID': 'Id'})
df4.to_csv("/content/drive/MyDrive/sarcasm/vel_Tamil_run2.csv", index=False)


921
5417


In [ ]:
df2

,Id,Labels
0,Id_01,Non-sarcastic
1,Id_02,Non-sarcastic
2,Id_03,Non-sarcastic
3,Id_04,Non-sarcastic
4,Id_05,Non-sarcastic
...,...,...
6333,Id_6334,Non-sarcastic
6334,Id_6335,Non-sarcastic
6335,Id_6336,Non-sarcastic
6336,Id_6337,Non-sarcastic


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
X = df['Text']
y = df['labels']
vectorizer = CountVectorizer(ngram_range=(1, 3))  # Adjust range as needed
x_train = vectorizer.fit_transform(X)
Y_train=y
x1=df1['Text']
x_test = vectorizer.transform(x1)
X_train, X_val, y_train, y_val = train_test_split(x_train, Y_train, test_size=0.3, random_state=42)

# Step 4: Fit the Model
model = SVC(kernel='linear')  # Linear kernel is often effective for text classification
model.fit(X_train, y_train)

# Step 5: Make Predictions
y_pred = model.predict(x_test)

df3=df1
df3['Labels']=y_pred
df3['Labels']=df3['Labels'].apply(lambda x:'Sarcastic' if x==1 else 'Non-sarcastic')
num_sarcastic = (df3['Labels'] == 'Sarcastic').sum()
print(num_sarcastic)
n = (df3['Labels'] == 'Non-sarcastic').sum()
print(n)
df3=df3.drop(columns=['Text'])
df3=df3.rename(columns={'ID': 'Id'})
df3.to_csv("/content/drive/MyDrive/sarcasm/vel_Tamil_run3.csv", index=False)


1101
5237


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Assuming df and df1 are already defined

# Step 1: Prepare data
X = df['Text']
y = df['labels']


# Step 2: Vectorize text data
vectorizer = CountVectorizer(ngram_range=(1, 3))  # Adjust range as needed
x_train = vectorizer.fit_transform(X)
y_train = y

# Check class distribution in training data
unique, counts = np.unique(y_train, return_counts=True)
print(f'Class distribution in y_train: {dict(zip(unique, counts))}')

# Vectorize the test data
x1 = df1['Text']
x_test = vectorizer.transform(x1)

# Step 3: Split Data
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

# Step 4: Train the model
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

# Step 5: Predict the labels of the validation set
y_val_pred = mnb.predict(X_val)

# Evaluate model performance on validation set
print(f'Validation accuracy: {accuracy_score(y_val, y_val_pred)}')
print(f'Classification report:\n{classification_report(y_val, y_val_pred)}')

# Step 6: Predict the labels of the test set
y_pred = mnb.predict(x_test)

# Create a new DataFrame with the predicted labels
df2 = df1.copy()
df2['Labels'] = y_pred
df2['Labels'] = df2['Labels'].apply(lambda x: 'Sarcastic' if x == 1 else 'Non-sarcastic')
df2=df2.drop(columns=['Text'])
df2=df2.rename(columns={'ID': 'Id'})
df2.to_csv("/content/drive/MyDrive/sarcasm/vel_Tamil_run1.csv", index=False)


Class distribution in y_train: {0: 21602, 1: 7820}
Validation accuracy: 0.7362637362637363
Classification report:
              precision    recall  f1-score   support

           0       0.89      0.73      0.80      6485
           1       0.50      0.74      0.60      2342

    accuracy                           0.74      8827
   macro avg       0.70      0.74      0.70      8827
weighted avg       0.79      0.74      0.75      8827

